<a href="https://colab.research.google.com/github/Young931127/ai-dream-analyzer/blob/main/final_project_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###**環境設置**
* **langchain & faiss**: 用於RAG與向量資料庫
* **groq & huggingface_hub**: 用於呼叫LLM與繪圖API
* **diffusers**: 用於生成圖片

In [ ]:
!pip install -q langchain langchain-community faiss-cpu sentence-transformers transformers huggingface_hub groq diffusers accelerate safetensors invisible_watermark

In [ ]:
import os
import json
import shutil
import torch
import gradio as gr
from groq import Groq
from diffusers import AutoPipelineForText2Image
from huggingface_hub import login ,InferenceClient
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

###**身分驗證與API KEY設定**
1.  從colab secret讀取API KEY
2.  若讀取失敗讓使用者手動輸入



In [ ]:
try:
  from google.colab import userdata
  os.environ['GROQ_API_KEY'] = userdata.get('Groq') #設定為環境變數供client 使用
  hf_token = userdata.get('HuggingFace')

  if hf_token:
    login(token = hf_token) #登入HuggingFace
  else :
    raise ValueError("No HF Token")

except:
  print("can not find Colab Secrets. Please enter secret key and token manually.")
  os.environ["GROQ_API_KEY"] = input("Please enter Groq API Key (gsk_...): ")
  print("Please enter Hugging Face Token:")
  login()

###**定義Embedding 模型**
*   使用Google 的`embeddinggemma-300m`模型將文字轉為向量





In [ ]:
class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
  def __init__(self, **kwargs):
    super().__init__(
      model_name="google/embeddinggemma-300m",
      encode_kwargs={"normalize_embeddings": True},
      **kwargs
    )

  def embed_documents(self, texts):
    # 針對文檔嵌入加上前綴
    texts = [f'title: none | text: {t}' for t in texts]
    return super().embed_documents(texts)

  def embed_query(self, text):
    # 官方檢索建議前綴
    return super().embed_query(f'task: search result | query: {text}')

embedding_model = EmbeddingGemmaEmbeddings()

###**讀入資料集 : 讀入RAG向量資料庫(FAISS)**
1.  從Google Drive 解壓縮夢境解析資料庫
2.  使用FAISS 套件載入解壓後的向量索引




In [ ]:
DRIVE_ZIP_PATH = "/content/drive/MyDrive/Dream/dream_db.zip"
EXTRACT_PATH = "loaded_db" # 暫存資料夾

if os.path.exists(DRIVE_ZIP_PATH):
  if os.path.exists(EXTRACT_PATH):
    shutil.rmtree(EXTRACT_PATH) # 清除舊資料，確保載入的是最新版

  print(f"正在解壓縮... ,Source :{DRIVE_ZIP_PATH}")
  shutil.unpack_archive(DRIVE_ZIP_PATH, EXTRACT_PATH)

  print("Loading FAISS vectorDB...")
  try:
    db_path = EXTRACT_PATH

    if "faiss_db" in os.listdir(EXTRACT_PATH):
      db_path = os.path.join(EXTRACT_PATH, "faiss_db")

    vectorstore = FAISS.load_local(
      folder_path=db_path,
      embeddings=embedding_model,
      allow_dangerous_deserialization=True
    )

    # 建立檢索器 (Retriever)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 1})  # k=1 代表只找最相關的那一筆資料
    print("資料庫載入成功")

  except Exception as e:
    print(f"資料庫載入失敗: {e}")
    print(f"目前解壓路徑內容: {os.listdir(EXTRACT_PATH)}")

else:
  print(f"error：can not find file {DRIVE_ZIP_PATH}")

###**定義System Prompt**

In [ ]:
DREAM_SYSTEM_TEMPLATE = """
  你是一位精通榮格心理學與超現實主義藝術的「夢境解讀師」。

  【參考資料】
  {knowledge}

  【任務】
  請根據使用者的夢境，結合上述參考知識，執行以下三項工作：
  1. 深度心理分析:拆解夢境中的關鍵象徵（人物、物品、情境），分析其隱含的潛意識情緒（如焦慮、渴望、恐懼）與現實生活的壓力源。
  2. 具體建議:根據分析出的壓力源，提供具體的行動建議（如心理轉念、放鬆練習、生活調整）。
  3. 視覺轉譯:將這個夢境的情緒與畫面，轉化為詳細的英文繪圖指令。

  【輸出格式規則】
  請務必直接輸出標準JSON格式，不要包含任何Markdown標記 (如 ```json)，包含以下三個欄位：

  1. "analysis": (字串) 深度心理分析。請指出夢中各個意象並解釋其意義，並推測這反映了使用者最近遇到什麼狀況(語氣溫暖專業，長度約100-200字)。
  2. "advice": (字串) 給使用者的具體建議。例如：「既然夢境顯示您最近壓力很大，建議您嘗試腹式呼吸法...」或「建議給自己安排一場無目的的散步」(長度約100-150字)。
  3. "prompt": (字串) 給AI繪圖模型的英文指令。
    i.繪圖指令必須包含風格詞：Surrealism (超現實主義), cinematic lighting (電影光效), 8k, highly detailed, dreamlike atmosphere.
    ii.描述必須具體，例如 "A giant tooth falling from the sky, cracking the ground..."
  【互動規則】
  1. 若使用者的描述太簡略，請不要分析，請用「自然中文」追問細節（例如：當下的情緒？顏色？）。
  2. 若資訊已充足(已經收集到夢境的「視覺畫面 (顏色/光影)」、「情緒氛圍」與「關鍵情節」，且經過了至少 1~2 輪的對話)，或使用者明確表示「請分析」、「就這樣」、「沒有其他細節」或「不知道」時，請立刻結束對話。請「只輸出」標準 JSON 格式（包含 analysis, advice, prompt）。
  3. 為了避免使用者疲乏，請不要追問超過「3個」問題，且問題盡量簡單好理解。

"""

### **AI Agent 核心邏輯：結合 RAG 知識檢索與對話記憶**
賦予AI夢境意象所代表的意涵相關知識 (RAG) 與短期對話記憶 (Context)
1. 感知 ：接收使用者最新的夢境描述。
2. 檢索 ：使用retriever 從向量資料庫中尋找相關的心理學理論 (RAG)。
3. 記憶 ：將Gradio 傳入的歷史對話(history) 轉換為LLM 讀得懂的格式，維持對話連貫性。
4. 生成 ：將「系統指令 + 參考知識 + 對話紀錄」打包，發送給Groq產生回應。





In [ ]:
client = Groq() # 初始化Groq 客戶端

def dream_interpreter_agent(user_message, history):
  # Retrieval
  print(f"正在檢索關於 '{user_message}' 的心理學知識...")
  try:
    docs = retriever.invoke(user_message) # 使用前面載入的retriever 來找資料
    knowledge = docs[0].page_content if docs else "無相關心理學資料"

  except Exception as e:
    knowledge = "資料庫檢索失敗"
    print(f"Debug: {e}")

  system_instruction = DREAM_SYSTEM_TEMPLATE.format(knowledge=knowledge)

  messages = [{"role" : "system", "content" : system_instruction }]

  # 將Gradio 的history ([[user, ai], [user, ai]...]) 轉換成API 格式
  for user_msg, ai_msg in history:
    if user_msg:
      messages.append({"role": "user", "content": user_msg})
    if ai_msg:
      messages.append({"role": "assistant", "content": ai_msg})

  messages.append({"role": "user", "content": user_message}) # 加入當前這句話

  #call LLM
  completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=messages,
    temperature=0.7 # 創意度
  )

  return completion.choices[0].message.content

###**圖象生成**
**兩種模式可替換**
*  **雲端伺服器運算** : 調用Huggiging Face Inference API，使用 `stable-diffusion-xl-base-1.0` 模型(Colab GPU 資源耗盡時的替代方案)
*  **使用Colab GPU運算** : 透過Diffusers在本地部署SDXL-Turbo 模型



In [ ]:
generation_mode = None  # 用來記錄目前是"local"還是 "api"
pipe = None
client_img = None
if torch.cuda.is_available() :
  print("偵測到GPU，正在嘗試載入模型(SDXL-Turbo)")

  try :
    pipe = AutoPipelineForText2Image.from_pretrained(
      "stabilityai/sdxl-turbo",
      torch_dtype=torch.float16,
      variant="fp16"
    )

    pipe.to("cuda")
    print("模型載入成功, 目前使用GPU繪圖")
    generation_mode = "local"

  except Exception as e:
    print(f"模型載入失敗: {e}")
    print("正在嘗試使用Hugging Face Inference API 繪圖...")

else :
  print("偵測到CPU，正在嘗試使用Hugging Face Inference API 繪圖...")

# 如果本地載入失敗，或是沒有GPU，就執行替代方案 : Hugging Face API雲端繪圖
if generation_mode is None :
  print("正在啟用Hugging Face Inference API 繪圖")

  try:
    # 透過Hugging Face Inference API 進行伺服器端運算，跑SDXL 1.0模型
    client_img = InferenceClient(model="stabilityai/stable-diffusion-xl-base-1.0", token=hf_token)
    print("已啟用：Hugging Face 雲端繪圖模式")
  except Exception as e:
    print(f"API 初始化失敗: {e}")
    print("錯誤：無法建立任何繪圖服務，請檢查Token或GPU 狀態。")

# 繪圖函式，根據情況自動切換
def generate_image(prompt_text) :
  if not prompt_text:
    print("提示詞為空")
    return None

  #使用本地GPU繪圖
  if generation_mode == "local":
    try:
      print(f"Local GPU 開始繪圖: {prompt_text[:50]}...")
       # SDXL Turbo 只需要1 step,guidance_scale 設0.0
      return pipe(prompt=prompt_text, num_inference_steps=1, guidance_scale=0.0).images[0]
    except Exception as e:
      print(f"繪圖失敗: {e}")
      return None

    # 使用Hugging Face API
  elif generation_mode == "api":
    try:
      print(f"Cloud API 開始繪圖: {prompt_text[:30]}...")
      return client_img.text_to_image(prompt_text)
    except Exception as e:
      print(f"API 繪圖失敗: {e}")
      return None
  else:
    print("統錯誤：沒有可用的繪圖模型。")
    return None

###Gradio 介面邏輯

In [ ]:
def chat_process(message, history):
  """
  message: 當前使用者輸入的字串
  history: 過去的對話紀錄List (Gradio 傳入)
  """
  ai_response = dream_interpreter_agent(message, history) # Call AI

  result_json = None
  # 判斷回覆是AI「追問」的問題還是「最終分析」
  try:

    # 找第一個中括號和最後一個中括號
    start_idx = ai_response.find('{')
    end_idx = ai_response.rfind('}')

    # 只有當同時找到兩個中括號時才嘗試解析
    if start_idx != -1 and end_idx != -1:
      # 擷取純 JSON 字串
      json_str = ai_response[start_idx : end_idx + 1]
      result_json = json.loads(json_str)

    # 若成功解析JSON 則代表分析完成
    if "analysis" in result_json:
      analysis = result_json.get("analysis", "無內容")
      advice = result_json.get("advice", "無內容")
      prompt = result_json.get("prompt", "")

      report = f"【深度心理分析】\n{analysis}\n\n【建議】\n{advice}" # 將「分析」跟「建議」組合成完整的分析報告
      chat_reply = "為您分析中..."

      return chat_reply, report, prompt, prompt # (聊天室回應, 分析報告, 繪圖指令, 觸發圖片生成)

  except (json.JSONDecodeError, TypeError):
    # 解析失敗，代表AI還在跟使用者對話
    pass

  # 若不是 JSON，直接把 AI 的回話丟回聊天室
  # 右側欄位回傳 gr.update() 代表「畫面不動」
  return ai_response, gr.update(), gr.update(), None

###Gradio 介面設計

In [ ]:
# 自定義 CSS：標題漸層與置中
custom_css = """
.dream-title {
    text-align: center;
    background: -webkit-linear-gradient(45deg, #6a11cb 0%, #2575fc 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-weight: bold;
    font-size: 3em;
    margin-bottom: 0.5em;
}
.dream-subtitle {
    text-align: center;
    font-size: 1.2em;
    color: #666;
    margin-bottom: 2em;
}
"""
with gr.Blocks(title="互動式夢境諮商室", theme=gr.themes.Soft(), css=custom_css) as demo:
  with gr.Column():
    gr.HTML("""
      <h1 class="dream-title">互動式夢境諮商室</h1>
      <p class="dream-subtitle">結合 RAG 知識庫與生成式 AI，解讀你的夢境並繪製潛意識畫面。</p>
    """)

  with gr.Row(equal_height=False):
    with gr.Column(scale=4):
      chatbot = gr.Chatbot(label="諮商紀錄", height=500, type="tuples")
      msg = gr.Textbox(label="請輸入您的夢境", placeholder="我夢到了... (按 Enter 發送)")
      clear = gr.Button("重新開始")

    with gr.Column(scale=3):
      gr.Markdown("### 分析結果")
      analysis_output = gr.Textbox(label="心理分析與建議", lines=12, interactive=False)

      gr.Markdown("### 夢境具象")
      image_output = gr.Image(label="AI 生成夢境圖像")
      prompt_output = gr.Accordion("查看繪圖指令 (Prompt)", open=False)
      with prompt_output:
        prompt_text = gr.Textbox(show_label=False)

  def respond(message, chat_history):
    bot_message, report, prompt, img_trigger = chat_process(message, chat_history)

    chat_history.append((message, bot_message)) # 更新歷史紀錄

    # 處理圖片生成
    gen_img = None
    if img_trigger:
      gen_img = generate_image(img_trigger)

    return "", chat_history, report, gen_img, prompt

  msg.submit(
    fn=respond,
    inputs=[msg, chatbot],
    outputs=[msg, chatbot, analysis_output, image_output, prompt_text]
  )

  #清除
  clear.click(lambda: (None, [], None, None, None), outputs=[msg, chatbot, analysis_output, image_output, prompt_text])

if __name__ == "__main__":
  demo.launch(
    share=True,
    debug=True
  )
